In [1]:
import numpy as np
import tensorflow as tf

C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
import tensorflow_datasets as tfds

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
tf.compat.v1.enable_eager_execution()
print(tf.__version__)
print(tf.executing_eagerly())

1.14.0
True


In [4]:
mnist_dataset,mnist_info = tfds.load(name='mnist',as_supervised=True,with_info=True)

In [5]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

In [6]:
num_validation_sample = 0.1*mnist_info.splits['train'].num_examples

In [7]:
num_validation_sample = tf.cast(num_validation_sample,tf.int64)

In [8]:
num_test_samples = mnist_info.splits['test'].num_examples

In [9]:
num_test_samples = tf.cast(num_validation_sample,tf.int64)

In [10]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image = image/255.
    return image,label
scale_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [11]:
buffer_size = 10000

In [12]:
shuffled_train_and_validation_data = scale_train_and_validation_data.shuffle(buffer_size)

In [13]:
validation_data = shuffled_train_and_validation_data.take(num_validation_sample)

In [14]:
train_data = shuffled_train_and_validation_data.skip(num_validation_sample)

In [15]:
BATCH_SIZE = 100

In [16]:
train_data = train_data.batch(BATCH_SIZE)

In [17]:
validation_data = validation_data.batch(num_validation_sample)

In [18]:
test_data = test_data.batch(num_test_samples)

In [19]:
validation_inputs,validation_targets = next(iter(validation_data))

In [20]:
input_size = 784
output_size = 10
hidden_layer_size = 100
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape =(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [21]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
num_epochs = 5
model.fit(train_data, epochs=num_epochs, validation_data = (validation_inputs,validation_targets),verbose=2)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


540/540 - 25s - loss: 0.3271 - acc: 0.9088 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/5
540/540 - 21s - loss: 0.1339 - acc: 0.9598 - val_loss: 0.1327 - val_acc: 0.9630
Epoch 3/5
540/540 - 24s - loss: 0.0963 - acc: 0.9718 - val_loss: 0.1098 - val_acc: 0.9670
Epoch 4/5
540/540 - 22s - loss: 0.0742 - acc: 0.9778 - val_loss: 0.0917 - val_acc: 0.9752
Epoch 5/5
540/540 - 20s - loss: 0.0593 - acc: 0.9817 - val_loss: 0.0689 - val_acc: 0.9813


In [23]:
test_loss,test_accuracy = model.evaluate(test_data)

      2/Unknown - 4s 2s/step - loss: 0.0835 - acc: 0.9748

In [24]:
print('test_loss:',test_loss,'\n','test_accuracy:',test_accuracy*100)

test_loss: 0.08353456482291222 
 test_accuracy: 97.47999906539917
